In [ ]:
!pip install ultralytics pyyaml -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.7 MB/s eta 0:00:00:00:0100:01


In [ ]:
import os
from ultralytics import RTDETR
import shutil
import random

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
!mkdir -p dataset/train/images dataset/train/labels dataset/val/images dataset/val/labels
#Создание структуры

In [ ]:
import os
import shutil
import random
from tqdm import tqdm

# --- 1. УКАЖИТЕ ПРАВИЛЬНЫЙ ИСХОДНЫЙ ПУТЬ ---
# Путь можно скопировать из левой панели "Data" в Kaggle.
SOURCE_DATA_ROOT = '/kaggle/input/computervisiondataset' # <--- ИЗМЕНИТЕ ЭТУ СТРОКУ!

# Пути к исходным папкам с изображениями и разметкой
source_image_dir = os.path.join(SOURCE_DATA_ROOT, 'images6/images6') #Смена частей разделенного датасета!
source_label_dir = os.path.join(SOURCE_DATA_ROOT, 'labels/labels')

# --- 2. ПУТИ НАЗНАЧЕНИЯ---
train_img_dir = '/kaggle/working/dataset/train/images'
train_lbl_dir = '/kaggle/working/dataset/train/labels'
val_img_dir = '/kaggle/working/dataset/val/images'
val_lbl_dir = '/kaggle/working/dataset/val/labels'

# Убедимся, что директории назначения существуют (на всякий случай)
os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(train_lbl_dir, exist_ok=True)
os.makedirs(val_img_dir, exist_ok=True)
os.makedirs(val_lbl_dir, exist_ok=True)

# --- 3. РАЗДЕЛЕНИЕ ФАЙЛОВ ---
print(f"Поиск изображений в: {source_image_dir}")

try:
    # Получаем список всех файлов изображений.
    images = [f for f in os.listdir(source_image_dir) if f.endswith(('.JPG', 'jpg'))]
    if not images:
        print("ОШИБКА: В исходной папке не найдено изображений. Проверьте путь и расширения файлов.")
    else:
        print(f"Найдено {len(images)} изображений.")
        random.shuffle(images)

        # Делим 85% на train, 15% на val
        split_ratio = 0.85
        split_idx = int(len(images) * split_ratio)
        train_imgs = images[:split_idx]
        val_imgs = images[split_idx:]

        # --- 4. ФУНКЦИЯ КОПИРОВАНИЯ ---
        def move_files(img_list, source_img_path, source_lbl_path, dest_img_path, dest_lbl_path):
            for img_name in tqdm(img_list, desc=f"Копирование в {os.path.basename(os.path.dirname(dest_img_path))}"):
                base_name = os.path.splitext(img_name)[0]
                label_name = f"{base_name}.txt"

                # Исходные пути
                src_img_file = os.path.join(source_img_path, img_name)
                src_lbl_file = os.path.join(source_lbl_path, label_name)

                # Пути назначения
                dest_img_file = os.path.join(dest_img_path, img_name)
                dest_lbl_file = os.path.join(dest_lbl_path, label_name)

                # Копируем, только если исходный файл существует
                if os.path.exists(src_img_file):
                    shutil.copy(src_img_file, dest_img_file)
                if os.path.exists(src_lbl_file):
                    shutil.copy(src_lbl_file, dest_lbl_file)

        # --- 5. ЗАПУСК КОПИРОВАНИЯ ---
        print("\nНачинаю копирование обучающих файлов...")
        move_files(train_imgs, source_image_dir, source_label_dir, train_img_dir, train_lbl_dir)

        print("\nНачинаю копирование валидационных файлов...")
        move_files(val_imgs, source_image_dir, source_label_dir, val_img_dir, val_lbl_dir)

        print(f"\n✅ Готово! Разделено: {len(train_imgs)} в train, {len(val_imgs)} в val.")
        print(f"Данные находятся в /kaggle/working/dataset")

except FileNotFoundError:
    print(f"КРИТИЧЕСКАЯ ОШИБКА: Не удалось найти исходную директорию: '{source_image_dir}'")
    print("Пожалуйста, убедитесь, что путь в переменной SOURCE_DATA_ROOT указан верно и датасет добавлен в ноутбук.")

Поиск изображений в: /kaggle/input/computervisiondataset/images6/images6
Найдено 4998 изображений.

Начинаю копирование обучающих файлов...


Копирование в train: 100%|██████████| 4248/4248 [03:28<00:00, 20.39it/s]



Начинаю копирование валидационных файлов...


Копирование в val: 100%|██████████| 750/750 [00:36<00:00, 20.38it/s]


✅ Готово! Разделено: 4248 в train, 750 в val.
Данные находятся в /kaggle/working/dataset


In [ ]:
#очистка рабочей директории
!rm -rf /kaggle/working/*

In [ ]:
import os
from tqdm import tqdm

# ==============================================================================
# ================================ НАСТРОЙКИ ===================================
# ==============================================================================

# Укажите путь к корневой папке вашего датасета в рабочей директории
DATASET_ROOT_PATH = '/kaggle/working/dataset'

# Укажите максимальный размер файла в байтах, который следует считать "пустым".
# Если ваши файлы весят 2 байта, порог в 5 байт их точно захватит.
EMPTY_FILE_THRESHOLD_BYTES = 5

# (Опционально) Расширения изображений для поиска парных файлов
IMAGE_EXTENSIONS = {'.JPG', '.jpg'}

# ==============================================================================
# ====================== ДАЛЬШЕ КОД МОЖНО НЕ ТРОГАТЬ ===========================
# ==============================================================================

class DataCleaner:
    def __init__(self, root_path, img_exts, threshold):
        self.root_path = root_path
        self.img_exts = img_exts
        self.threshold = threshold
        self.files_to_delete = {'labels': [], 'images': []}
        self.deleted_count = 0

    def find_problematic_files(self):
        print(f"🚀 Поиск 'почти пустых' файлов (размер <= {self.threshold} байт) в: {self.root_path}")
        for split in ['train', 'val']:
            print(f"\n🔍 Сканирую раздел: '{split}'...")
            label_dir = os.path.join(self.root_path, split, 'labels')
            image_dir = os.path.join(self.root_path, split, 'images')

            if not os.path.exists(label_dir):
                print(f"Папка {label_dir} не найдена, пропускаю.")
                continue

            for filename in tqdm(os.listdir(label_dir), desc=f"Анализ '{split}/labels'"):
                if not filename.endswith('.txt'):
                    continue

                label_path = os.path.join(label_dir, filename)

                # --- ИЗМЕНЕНИЕ ЗДЕСЬ ---
                # Проверяем, что размер файла меньше или равен нашему порогу
                if os.path.getsize(label_path) <= self.threshold:
                    self.files_to_delete['labels'].append(label_path)

                    # Ищем соответствующее изображение для удаления
                    base_name = os.path.splitext(filename)[0]
                    found_image = False
                    for ext in self.img_exts:
                        image_path = os.path.join(image_dir, base_name + ext)
                        if os.path.exists(image_path):
                            self.files_to_delete['images'].append(image_path)
                            found_image = True
                            break
                    if not found_image:
                        print(f"⚠️ Не найдено парное изображение для проблемной разметки: {label_path}")

    def delete_files(self):
        total_files = len(self.files_to_delete['labels']) + len(self.files_to_delete['images'])
        if total_files == 0:
            print(f"\n✅ Не найдено файлов разметки размером <= {self.threshold} байт. Удалять нечего.")
            return

        print(f"\n🔥 Найдено {len(self.files_to_delete['labels'])} проблемных файлов разметки и {len(self.files_to_delete['images'])} соответствующих изображений.")

        user_input = input(f"Вы уверены, что хотите удалить {total_files} файлов? (введите 'да' для подтверждения): ")

        if user_input.lower() == 'да':
            print("\nНачинаю удаление...")
            # Удаляем файлы разметки
            for f_path in tqdm(self.files_to_delete['labels'], desc="Удаление разметок"):
                try:
                    os.remove(f_path)
                    self.deleted_count += 1
                except OSError as e:
                    print(f"Ошибка при удалении файла {f_path}: {e}")

            # Удаляем изображения
            for f_path in tqdm(self.files_to_delete['images'], desc="Удаление изображений"):
                try:
                    os.remove(f_path)
                    self.deleted_count += 1
                except OSError as e:
                    print(f"Ошибка при удалении файла {f_path}: {e}")

            print(f"\n✅ Готово! Удалено {self.deleted_count} файлов.")
        else:
            print("\nОтмена. Файлы не были удалены.")

# --- ЗАПУСК ПРОЦЕССА ОЧИСТКИ ---
cleaner = DataCleaner(DATASET_ROOT_PATH, IMAGE_EXTENSIONS, EMPTY_FILE_THRESHOLD_BYTES)
cleaner.find_problematic_files()
cleaner.delete_files()


🚀 Поиск 'почти пустых' файлов (размер <= 5 байт) в: /kaggle/working/dataset

🔍 Сканирую раздел: 'train'...


Анализ 'train/labels': 100%|██████████| 4248/4248 [00:00<00:00, 137778.70it/s]



🔍 Сканирую раздел: 'val'...


Анализ 'val/labels': 100%|██████████| 750/750 [00:00<00:00, 126076.23it/s]



🔥 Найдено 358 проблемных файлов разметки и 358 соответствующих изображений.


Вы уверены, что хотите удалить 716 файлов? (введите 'да' для подтверждения):  да



Начинаю удаление...


Удаление изображений: 100%|██████████| 358/358 [00:00<00:00, 2917.07it/s]


✅ Готово! Удалено 716 файлов.


In [ ]:
#Копирование весов
!cp /kaggle/input/modeltrain/version5/version5/best.pt /kaggle/working/

In [ ]:
#Пути
WORK_DIR = "/kaggle/working"
DATA_DIR = f"{WORK_DIR}/dataset"
YAML_PATH = f"{WORK_DIR}/dataset.yaml"

In [ ]:
#Создание dataset.yaml
with open(YAML_PATH, "w") as f:
    f.write(f"""
path: {DATA_DIR}
train: /kaggle/working/dataset/train/images
val: /kaggle/working/dataset/val/images
nc: 1
names: ['person']
""")